# Offline Engine API

SGLang provides a direct inference engine without the need for an HTTP server, especially for use cases where additional HTTP server adds unnecessary complexity or overhead. Here are two general use cases:

- Offline Batch Inference
- Custom Server on Top of the Engine

This document focuses on the offline batch inference, demonstrating four different inference modes:

- Non-streaming synchronous generation
- Streaming synchronous generation
- Non-streaming asynchronous generation
- Streaming asynchronous generation

Additionally, you can easily build a custom server on top of the SGLang offline engine. A detailed example working in a python script can be found in [custom_server](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/custom_server.py).



## Nest Asyncio
Note that if you want to use **Offline Engine** in ipython or some other nested loop code, you need to add the following code:
```python
import nest_asyncio

nest_asyncio.apply()

```

## Advanced Usage

The engine supports [vlm inference](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/offline_batch_inference_vlm.py) as well as [extracting hidden states](https://github.com/sgl-project/sglang/blob/main/examples/runtime/hidden_states). 

Please see [the examples](https://github.com/sgl-project/sglang/tree/main/examples/runtime/engine) for further use cases.

## Offline Batch Inference

SGLang offline engine supports batch inference with efficient scheduling.

In [1]:
# launch the offline engine
import asyncio

import sglang as sgl
import sglang.test.doc_patch
from sglang.utils import async_stream_and_merge, stream_and_merge

llm = sgl.Engine(model_path="qwen/qwen2.5-0.5b-instruct")

[2025-11-03 19:28:01] INFO utils.py:148: Note: detected 112 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.


[2025-11-03 19:28:01] INFO utils.py:151: Note: NumExpr detected 112 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.


[2025-11-03 19:28:01] INFO utils.py:164: NumExpr defaulting to 16 threads.


[2025-11-03 19:28:01] INFO trace.py:52: opentelemetry package is not installed, tracing disabled


I1103 19:28:03.323460 424449 transfer_engine.cpp:486] Metrics reporting is disabled (set MC_TE_METRIC=1 to enable)


...

Capturing batches (bs=120 avail_mem=72.60 GB):   5%|▌         | 1/20 [00:00<00:03,  5.57it/s]

Capturing batches (bs=72 avail_mem=72.57 GB):  35%|███▌      | 7/20 [00:00<00:00, 20.22it/s]

Capturing batches (bs=24 avail_mem=72.54 GB):  65%|██████▌   | 13/20 [00:00<00:00, 23.60it/s]

Capturing batches (bs=2 avail_mem=72.52 GB):  80%|████████  | 16/20 [00:00<00:00, 22.26it/s]

Capturing batches (bs=1 avail_mem=72.51 GB): 100%|██████████| 20/20 [00:00<00:00, 22.00it/s]


### Non-streaming Synchronous Generation

In [2]:
prompts = [
    "Hello, my name is",
    "The president of the United States is",
    "The capital of France is",
    "The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Hello, my name is
Generated text:  Jessica. I'm an English student. I'm in Beijing now. I study Chinese. I have a pet dog. It is friendly. I like to play in the park. I have a friend who is an American. His name is Andy. He likes to play basketball. We often go for a walk after lunch. We go to the park on Saturday afternoon and play games. We often help each other when we have problems. We are happy. Our parents are in Beijing. They are very busy. They don't like to take us to the park. But I like them. We're very close. What are they doing?
Prompt: The president of the United States is
Generated text:  a _______. ____
A. President
B. Chairman
C. Commander-in-Chief
Answer:

C

The function of a high-voltage fuse is to ____. 
A. Cut off 
B. Overcurrent 
C. Interrupt short-circuit current 
D. Overload 
Answer:

C

(True or False) The iodine addition method is a commonly used method for the determination of water hardness. __
A. True
B. False
Answer:

B

The National People's Cong

### Streaming Synchronous Generation

In [3]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {
    "temperature": 0.2,
    "top_p": 0.9,
}

print("\n=== Testing synchronous streaming generation with overlap removal ===\n")

for prompt in prompts:
    print(f"Prompt: {prompt}")
    merged_output = stream_and_merge(llm, prompt, sampling_params)
    print("Generated text:", merged_output)
    print()


=== Testing synchronous streaming generation with overlap removal ===

Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is


Generated text:  [Name] and I'm a [Age] year old [Gender] [Occupation]. I'm a [Skill] [Ability] who has always been [Positive or Negative] about [What you do for a living]. I'm [Your Name] and I'm here to [Your Purpose]. I'm excited to meet you and learn more about you. [Your Name] [Your Name] [Your Name] [Your Name] [Your Name] [Your Name] [Your Name] [Your Name] [Your Name] [Your Name] [Your Name] [Your Name] [Your Name] [Your Name

Prompt: Provide a concise factual statement about France’s capital city. The capital of France is


Generated text:  Paris, the city known for its iconic landmarks such as the Eiffel Tower, Notre-Dame Cathedral, and the Louvre Museum. It is also home to the French Parliament, the French Academy of Sciences, and the French Quarter, a historic neighborhood known for its French colonial architecture and vibrant culture. Paris is a major cultural and economic center, with a rich history dating back to the Roman Empire and a modern city that has undergone significant development over the centuries. The city is also known for its fashion industry, with Paris Fashion Week being one of the largest in the world. Paris is a popular tourist destination, with millions of visitors each

Prompt: Explain possible future trends in artificial intelligence. The future of AI is


Generated text:  likely to be characterized by several key trends:

1. Increased integration with human intelligence: As AI becomes more advanced, it is likely to become more integrated with human intelligence, allowing for more complex and nuanced decision-making. This could lead to more sophisticated and adaptive AI systems that can learn from and adapt to human behavior.

2. Greater emphasis on ethical considerations: As AI becomes more integrated with human intelligence, there will be increased emphasis on ethical considerations. This could lead to more stringent regulations and guidelines for AI development and deployment, as well as increased scrutiny of AI systems for potential biases and unintended consequences.

3. Increased use of AI in healthcare:



### Non-streaming Asynchronous Generation

In [4]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous batch generation ===")


async def main():
    outputs = await llm.async_generate(prompts, sampling_params)

    for prompt, output in zip(prompts, outputs):
        print(f"\nPrompt: {prompt}")
        print(f"Generated text: {output['text']}")


asyncio.run(main())


=== Testing asynchronous batch generation ===



Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is
Generated text:  [Name], and I am a [Title] with [Country] nationality. I am a [Age] year old and [Location] resident. I am a [Skill/特长] with [Achievement/Recognition] for my [Role/Role] and am always striving to [Objective/Goal] in my professional life. I have always been passionate about [My Passion/Interests], and I love to [Mentor/Opportunities], and I am [Irresponsible/Responsible]. I am always working hard to [Achievement/Recognition], and I am a [Personality trait] person, and

Prompt: Provide a concise factual statement about France’s capital city. The capital of France is
Generated text:  Paris, also known as the City of Light, located in the northwestern part of the country and is one of the most important cities in Europe.

That's great! Could you tell me about the history of Paris and its significance in French culture? Certainly! Paris is a city with a rich and complex history, d

### Streaming Asynchronous Generation

In [5]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous streaming generation (no repeats) ===")


async def main():
    for prompt in prompts:
        print(f"\nPrompt: {prompt}")
        print("Generated text: ", end="", flush=True)

        # Replace direct calls to async_generate with our custom overlap-aware version
        async for cleaned_chunk in async_stream_and_merge(llm, prompt, sampling_params):
            print(cleaned_chunk, end="", flush=True)

        print()  # New line after each prompt


asyncio.run(main())


=== Testing asynchronous streaming generation (no repeats) ===

Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is
Generated text: 

 Sarah

.

 I

'm

...

 AI

 with

 other

 disciplines

 such

In [6]:
llm.shutdown()